In [1]:
# extract stable audio vae latent from audio and save, with same file name nut .npy, and directory + _latent
from vocos import get_voco

voco = get_voco("oobleck")
voco.to('cuda')

OobleckVoco(
  (feature_extractor): OobleckFeatures(
    (encoder): OobleckEncoder(
      (conv1): Conv1d(2, 128, kernel_size=(7,), stride=(1,), padding=(3,))
      (block): ModuleList(
        (0): OobleckEncoderBlock(
          (res_unit1): OobleckResidualUnit(
            (snake1): Snake1d()
            (conv1): Conv1d(128, 128, kernel_size=(7,), stride=(1,), padding=(3,))
            (snake2): Snake1d()
            (conv2): Conv1d(128, 128, kernel_size=(1,), stride=(1,))
          )
          (res_unit2): OobleckResidualUnit(
            (snake1): Snake1d()
            (conv1): Conv1d(128, 128, kernel_size=(7,), stride=(1,), padding=(9,), dilation=(3,))
            (snake2): Snake1d()
            (conv2): Conv1d(128, 128, kernel_size=(1,), stride=(1,))
          )
          (res_unit3): OobleckResidualUnit(
            (snake1): Snake1d()
            (conv1): Conv1d(128, 128, kernel_size=(7,), stride=(1,), padding=(27,), dilation=(9,))
            (snake2): Snake1d()
            (c

In [4]:
import torch

picks = torch.load("text_alignment_v3_0414_65000.ckpt")

/tmp/ipykernel_1651189/928391420.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  picks = torch.load("text_alignment_v3_0414_65000.ckpt")


In [13]:
picks['hyper_parameters']

{'dim': 1024,
 'depth': 24,
 'heads': 16,
 'attn_dropout': 0.0,
 'ff_dropout': 0.1,
 'kernel_size': 31,
 'voco_type': 'oobleck',
 'max_audio_len': 400,
 'optimizer': 'Adam',
 'lr': 0.0002,
 'scheduler': 'linear_warmup_decay',
 'use_torchode': True,
 'torchdiffeq_ode_method': 'midpoint',
 'torchode_method_klass': 'tsit5',
 'max_steps': 300000,
 'text_repo_id': 'google/flan-t5-base'}

In [4]:
import librosa
import torchaudio
from einops import rearrange
import re
import os
from tqdm import tqdm
import numpy as np
import torch

audios_dir = '/workspace/AVE_Dataset/AVE_audio/'
save_dir = '/workspace/AVE_Dataset/AVE_audio_latent/'
audio_files = os.listdir(audios_dir)
os.makedirs(save_dir, exist_ok=True)

for af in tqdm(audio_files):
    audio, sr = librosa.load(audios_dir + af, sr=voco.sampling_rate, mono=False)
    audio = torch.tensor(audio)
    if len(audio.shape) == 1:
        audio = torch.stack([audio, audio])
    audio = rearrange(audio, 'c n -> n c')
    
    lt = voco.encode(audio.unsqueeze(dim=0).to('cuda')).squeeze()[:215, :].cpu().float().numpy()
    save_path = re.sub("\.wav", ".npy", save_dir+af)
    np.save(save_path, lt)

100%|██████████| 4097/4097 [09:33<00:00,  7.14it/s]


In [5]:
assd = (0, 1, 2)
assd[:-1]

(0, 1)